In [17]:
import sys
import os
from pathlib import Path
try:
    import pyvips  # type: ignore
    _HAS_PYVIPS = True
except Exception:
    pyvips = None
    _HAS_PYVIPS = False

PROJECT_ROOT = Path("/home/users/ntu/lizh0106/scratch/nscc_work/Baseline_models")
os.chdir(PROJECT_ROOT)
#sys.path.insert(0, str(PROJECT_ROOT))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

from src.config import load_paths, load_params
from src.data.features import prepare_features
from src.models.train import cross_validate, predict_external
from src.evaluation.grading import agg_from_tiles
from src.evaluation.metrics import evaluate_slide_predictions

'/home/users/ntu/lizh0106'